This is my notebook for the competition LECR :)

I'll do a simple data analysis, and try models like SBERT and SimCSE.

I'm new to Kaggle. Please leave your comments and I'll make improvements :)

# Introduction
This is where I briefly introduce the data provided by the host.

And I will mention some tips for this competition.

1. GOAL: The goal of this competition is to match content to the specific topic *(WARNING: THERE MIGHT BE MORE THAN ONE CORRELATED CONTENT)*.


2. DATASET:
    * `topics.csv`:  Contains a row for each topic in the dataset.
    * `content.csv`: Contains a row for each content item in the dataset.
    * `correlations.csv`: Contains `topic_id` with the correlated `content_ids`.
    * `sample_submission.csv`: A submission file in the correct format *(WARNING: YOU SHOULD ONLY SUBMIT PREDICTIONS FOR THOSE TOPICS LISTED IN THIS FILE)*.


3. EVALUATION:
    * LECR uses MEAN F2 SCORE for evaluation.
    * Since it's a CODE COMPETITION, the actual test set contains additional topics and content items. In the public version, the sample test data are drawn from the training set.


4. TIPS FROM THE HOST:
    * **Context matters! Explore the tree**: This is because other nodes in the topic tree may contain more semantic context than the given topic.
    * **Narrow down by language**: Most of the time the language of a topic will match the language of its correlated content. You can use this feature to narrow down or give priority.
    * **Focus on aligned and supplemental for performance**: The testing dataset does not contain any topics from `source` channels. So it's important to focus on these kind of data to achieve a better performance.
    * **Balance the semantics of `title`, `description`, and `text`**: It's important to carefully weight these fields because the semantic information they contain vary across topics and content items.
    * **Disregard `copyright_holder` for training purposes**: This field is blanked out in the testing data. So don't use it in the training phase.
    * **Restructure correlations for efficiency**: Considering the efficiency, you may need to reconstruct the correlations in `correlations.csv`.

# Exploratoty Data Analysis
This is where I do some simple data analysis.

## Before Actual Work

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os

DATA_PATH = "/kaggle/input/learning-equality-curriculum-recommendations/"

In [2]:
content = pd.read_csv(DATA_PATH + 'content.csv')
topics = pd.read_csv(DATA_PATH + 'topics.csv')
correlations = pd.read_csv(DATA_PATH + 'correlations.csv')

## Topics

In [3]:
topics.head()

,id,title,description,channel,category,level,language,parent,has_content
0,t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True
1,t_000095e03056,Unit 3.3 Enlargements and Similarities,NaN,b3f329,aligned,2,en,t_aa32fb6252dc,False
2,t_00068291e9a4,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True
3,t_00069b63a70a,Transcripts,NaN,6e3ba4,source,3,en,t_4054df11a74e,True
4,t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True


* Each topic has 9 fields.

# Modeling
This is where I try models like SBERT and SimCSE.

I'll explain why I choose these two models, and provide some examples to illustrate the usage.

Considering the GOAL of this competition (matching content items to the given topic), this is actually a Semantic Textual Similarity task.

[SBERT](https://www.sbert.net/), or Sentence-BERT is a novel framework for computing sentence/text embeddings for more than 100 languages **(multilingual)**. These embeddings can then be used in tasks like semantic similarity. It's easy to use with the help of Hugging Face. SBERT acutally uses a Siamese Network Structure, which is way **faster** than the usual Cross-Encoder Structure.

[SimCSE](https://github.com/princeton-nlp/SimCSE) adds the idea of contrastive learning to SBERT. It achieves **SOTA** performance on **unsupervised** learning task.

In [4]:
# This is a simple example about how to use SBERT
# SentenceTransformers implements the MultipleNegativesRankingLoss, which makes training with SimCSE trivial
# Click the links above to see more details
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 652.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=2940c978d102051ca41de7977fe772e9a7526f548d6c06422d2abb9de62db346
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [5]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2') # fast but also achieves good result

# lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome']
sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

# compute embeddings
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

# compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

# output the pairs with scores
for i in range(len(sentences1)):
    print("{} \t {} \t score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The cat sits outside 	 The dog plays in the garden 	 score: 0.2838
A man is playing guitar 	 A woman watches TV 	 score: -0.0327
The new movie is awesome 	 The new movie is so great 	 score: 0.8939


# Evaluation
This is where I explain the F2 SCORE.

When we evaluate a mdoel, we often use metrics like Precision and Recall. However, it's hard to achieve good results on both scores, so people should trade off between Precision and Recall. That's when people came up with a measurement called the **F-Score**. 

The F-Score is a the harmonic mean of a system's precision and recall values. It can be calculated by the following formula:

$$F\text-Score = (1+\beta^2) \cdot \frac{Precision \cdot Recall}{\beta^2 \cdot Precision+Recall}$$

If we set $\beta = n$, the $F\text-Score$ is now the $F_n\text-Score$. This means that if we set $\beta = 2$, we'll get the formula of **F2-Score**.